<a href="https://colab.research.google.com/github/krishghiya/Coronavirus-Diagnosis-ML/blob/master/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

os.environ['KAGGLE_USERNAME'] = 'sunstorm7'
os.environ['KAGGLE_KEY'] = 'b893e1f97505b1dcc3ff84703893e950'

# !kaggle datasets download -d arashnic/covid19-case-surveillance-public-use-dataset
# !kaggle datasets download -d tanmoyx/covid19-patient-precondition-dataset --unzip
# !kaggle datasets download -d cprete/covid19-open-datasets-for-brazil --unzip
# !kaggle datasets download -d nhntran/vietnam-covid19-patient-dataset --unzip

# county_data = keras.utils.get_file('us-counties.csv', 'https://github.com/nytimes/covid-19-data/raw/master/live/us-counties.csv')
# !kaggle datasets download -d rupsikaushik/covid19-patientlevel-data
# !kaggle datasets download -d shirmani/characteristics-corona-patients --unzip
!kaggle datasets download -d hemanthhari/symptoms-and-covid-presence --unzip

  0% 0.00/25.8k [00:00<?, ?B/s]
100% 25.8k/25.8k [00:00<00:00, 21.4MB/s]


In [34]:
ls

'Covid Dataset.csv'   sample_data/


In [35]:
# us_case_df = pd.read_csv('covid19-case-surveillance-public-use-dataset.zip') 
# mexico_df = pd.read_csv('covid.csv')
# brazil_sars_df = pd.read_csv('Flu-Like Syndrome/dados-nacional_01_06.csv', sep=";", error_bad_lines=False)
# brazil_sars_df = pd.read_csv('SARS_01-06/SRAG_01-06.csv')
# vietnam_df = pd.read_csv('Vietnam_COVID-19_patient_data_May10_2020.csv')

# county_df = pd.read_csv(county_data)
# china_df = pd.read_csv('covid19-patientlevel-data.zip')
# multi_loc_df = pd.read_csv('Characteristics_Corona_patients_version_6 -06122020.csv')
who_df = pd.read_csv('Covid Dataset.csv')

In [36]:
# mexico_df = mexico_df[['sex', 'pneumonia', 'age', 'pregnancy', 'diabetes', 'copd', 'asthma', 'hypertension', 'other_disease', 'cardiovascular', 'obesity', 
#                        'renal_chronic', 'tobacco', 'contact_other_covid', 'covid_res']]
# us_case_df = us_case_df[['sex', 'age_group', 'Race and ethnicity (combined)']].dropna()
# vietnam_df = vietnam_df[['Gender', 'Age', 'Detail Symptoms When Confirmed (clean up)']]

In [37]:
# china_df = china_df[['gender', 'age', 'symptom']]
# china_df = china_df.loc[china_df['symptom'].notna()]

In [38]:
# county_df = county_df[['county', 'state', 'cases', 'deaths']]

In [39]:
who_df = who_df.iloc[:, np.r_[:13, 20]]
who_df = who_df == 'Yes'
# who_df = who_df.where(who_df, 0).where(who_df != True, 1)
# who_df = who_df.astype('int')

In [8]:
# multi_loc_df = multi_loc_df.iloc[:, np.r_[25, 21, 23, 22, 26, 33, 37, 27, 35, 30, 31, 29, 34]]
# multi_loc_df['COVID-19'] = np.ones(len(multi_loc_df))
# multi_loc_df.columns = who_df.columns

In [9]:
# china_df.info()

In [10]:
# multi_loc_df.info()

In [40]:
who_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5434 entries, 0 to 5433
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Breathing Problem     5434 non-null   bool 
 1   Fever                 5434 non-null   bool 
 2   Dry Cough             5434 non-null   bool 
 3   Sore throat           5434 non-null   bool 
 4   Running Nose          5434 non-null   bool 
 5   Asthma                5434 non-null   bool 
 6   Chronic Lung Disease  5434 non-null   bool 
 7   Headache              5434 non-null   bool 
 8   Heart Disease         5434 non-null   bool 
 9   Diabetes              5434 non-null   bool 
 10  Hyper Tension         5434 non-null   bool 
 11  Fatigue               5434 non-null   bool 
 12  Gastrointestinal      5434 non-null   bool 
 13  COVID-19              5434 non-null   bool 
dtypes: bool(14)
memory usage: 74.4 KB


In [12]:
# multi_loc_df.isnull().sum() * 100 / len(multi_loc_df)

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(who_df.drop('COVID-19', axis=1), who_df['COVID-19'], test_size=0.1, random_state=42)

In [50]:
from sklearn.ensemble import RandomForestClassifier

ran_clf = RandomForestClassifier(n_estimators=100, random_state=42)
ran_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [51]:
from sklearn.metrics import accuracy_score, f1_score

y_pred = ran_clf.predict(X_test)
print('Accuracy score:', accuracy_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred))

Accuracy score: 0.9852941176470589
F1 score: 0.9906542056074767
